In [1]:
import warnings
warnings.filterwarnings('ignore')

import argparse
import time

import numpy as np
from scipy.sparse import coo_matrix, csc_matrix
from scipy.sparse.linalg import svds

import torch
from input_data import load_data
from preprocessing import general_train_test_split_edges,biased_train_test_split_edges, bidirectional_train_test_split_edges
from layers import DirectedGCNConvEncoder, DirectedInnerProductDecoder
from layers import SingleLayerDirectedGCNConvEncoder
from models import DirectedGAE

from layers import InnerProductDecoder
from layers import DummyEncoder, DummyPairEncoder

from models import GAE

In [2]:
parser = argparse.ArgumentParser()

parser.add_argument('--dataset',
                    nargs= '?',
                    default='citeseer',
                    type=str)
parser.add_argument('--task',
                    nargs= '?',
                    default='task_3',
                    type=str)
parser.add_argument('--model',
                    nargs= '?',
                    default='digae',
                    type=str)
parser.add_argument('--learning_rate',
                    nargs= '?',
                    default=0.01,
                    type=float)
parser.add_argument('--epochs',
                    nargs= '?',
                    default=200,
                    type=int)
parser.add_argument('--hidden',
                    nargs= '?',
                    default=64,
                    type=int)
parser.add_argument('--dimension',
                    nargs= '?',
                    default=32,
                    type=int)
parser.add_argument('--alpha',
                    nargs= '?',
                    default=1.0,
                    type=float)
parser.add_argument('--beta',
                    nargs= '?',
                    default=0.0,
                    type=float)
parser.add_argument('--nb_run',
                    nargs= '?',
                    default=10,
                    type=int)
parser.add_argument('--prop_val',
                    nargs= '?',
                    default=5.0,
                    type=float)
parser.add_argument('--prop_test',
                    nargs= '?',
                    default=10.0,
                    type=float)
parser.add_argument('--verbose',
                    nargs= '?',
                    default=True,
                    type=bool)

parser.add_argument('--self_loops',
                    nargs= '?',
                    default=True,
                    type=bool)
parser.add_argument('--adaptive',
                    nargs= '?',
                    default=False,
                    type=bool)
parser.add_argument('--feature_vector_type',
                    nargs= '?',
                    const=None)
parser.add_argument('--feature_vector_size',
                    nargs= '?',
                    const=None,
                    type=int)
parser.add_argument('--directed',
                    nargs= '?',
                    default=True,
                    type=bool)
parser.add_argument('--logfile',
                    nargs='?',
                    default='logs.json',
                    type=str)
parser.add_argument('--validate',
                    nargs='?',
                    default=False,
                    type=bool)

args,_ = parser.parse_known_args()

In [3]:
def train_single():
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    loss.backward()
    optimizer.step()
    return float(loss)


def test_single(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)

def dummy_train_single():
    model.train()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    return float(loss)

def dummy_test_single(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)
    

def train_pair():
    model.train()
    optimizer.zero_grad()
    s, t = model.encode(u, v, train_pos_edge_index)
    loss = model.recon_loss(s, t, train_pos_edge_index)
    loss.backward()
    optimizer.step()
    return float(loss)

def test_pair(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        s, t = model.encode(u, v, train_pos_edge_index)
    return model.test(s, t, pos_edge_index, neg_edge_index)

def dummy_train_pair():
    model.train()
    s, t = model.encode(u, v, train_pos_edge_index)
    loss = model.recon_loss(s, t, train_pos_edge_index)
    return float(loss)

def dummy_test_pair(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        s, t = model.encode(u, v, train_pos_edge_index)
    return model.test(s, t, pos_edge_index, neg_edge_index)

def identity_features(data, k=None):
    num_nodes = data.x.size(0)
    x = torch.eye(num_nodes)
    u_hat = x.clone()
    v_hat = x.clone()
    return u_hat, v_hat

In [4]:
dummy_run = False
 
if args.verbose:
    print("Loading data...")

device     = 'cpu'
directed   = args.directed

dataset_name    = args.dataset
out_channels    = args.dimension
hidden_channels = args.hidden

compute_features = identity_features

feature_vector_size = args.feature_vector_size

if args.model in ['gae','source_target']:
    train_func = train_single
    test_func  = test_single
elif args.model in ['dummy']:
    train_func = dummy_train_single
    test_func  = dummy_test_single
elif args.model in ['dummy_pair']:
    train_func = dummy_train_pair
    test_func  = dummy_test_pair
else:
    train_func = train_pair
    test_func  = test_pair

val_ratio  = args.prop_val / 100.
test_ratio = args.prop_test / 100.

loaded_data = load_data(dataset_name, directed=directed)

Loading data...


In [5]:
mean_roc  = []
mean_ap   = []
mean_time = []

for i in range(args.nb_run):
    if args.verbose:
        print("Masking test edges...")

    if args.task == 'task_1':
        data = loaded_data.clone()
        data.train_mask = data.val_mask = data.test_mask = data.y = None
        data            = general_train_test_split_edges(data,
                                                         val_ratio=val_ratio,
                                                         test_ratio=test_ratio,
                                                         directed=directed)
    elif args.task == 'task_2':
        data = loaded_data.clone()
        data.train_mask = data.val_mask = data.test_mask = data.y = None
        data            = biased_train_test_split_edges(data,
                                                        val_ratio=val_ratio,
                                                        test_ratio=test_ratio,
                                                        directed=directed)
    elif args.task == 'task_3':
        data = loaded_data.clone()
        data.train_mask = data.val_mask = data.test_mask = data.y = None
        data            = bidirectional_train_test_split_edges(data,
                                                               val_ratio=val_ratio,
                                                               test_ratio=test_ratio,
                                                               directed=directed)     
    else:
        raise ValueError('Undefined task!')

    data = data.to(device)
    train_pos_edge_index = data.train_pos_edge_index.to(device)
    if args.validate is True:
        test_pos_edge_index  = data.val_pos_edge_index.to(device)
        test_neg_edge_index  = data.val_neg_edge_index.to(device)
    else:
        test_pos_edge_index  = data.test_pos_edge_index.to(device)
        test_neg_edge_index  = data.test_neg_edge_index.to(device)

    in_channels          = data.x.shape[1]

    in_channels = data.x.size(0)
    u, v = compute_features(data, in_channels)
    data.u = u
    data.v = v
    data.x = torch.cat([data.u, data.v], dim=1)
    
    u = data.u.to(device)
    v = data.v.to(device)
    print(u.shape, v.shape)

    x = data.x.to(device)

    if args.model == 'digae':
        encoder = DirectedGCNConvEncoder(in_channels, hidden_channels, out_channels, alpha=args.alpha, beta=args.beta,
                                         self_loops=args.self_loops,
                                         adaptive=args.adaptive)
        decoder = DirectedInnerProductDecoder()
        model   = DirectedGAE(encoder, decoder)
        model   = model.to(device)

    elif args.model == 'digae_single_layer':
        encoder = SingleLayerDirectedGCNConvEncoder(in_channels, out_channels, alpha=args.alpha, beta=args.beta,
                                                    self_loops=args.self_loops,
                                                    adaptive=args.adaptive)
        decoder = DirectedInnerProductDecoder()
        model   = DirectedGAE(encoder, decoder)
        model   = model.to(device)

    elif args.model == 'dummy':
        encoder = DummyEncoder()
        decoder = InnerProductDecoder()
        model   = GAE(encoder, decoder)
        model   = model.to(device)
        dummy_run = True

    elif args.model == 'dummy_pair':
        encoder = DummyPairEncoder()
        decoder = DirectedInnerProductDecoder()
        model   = DirectedGAE(encoder, decoder)
        model   = model.to(device)
        dummy_run =True
    else:
        raise ValueError('Undefined model!')

    optimizer = torch.optim.Adam(model.parameters(), lr=args.learning_rate, weight_decay=5e-4)

    if args.verbose:
        print("Training...")

    # Flag to compute total running time
    t_start = time.time()
    for epoch in range(args.epochs):
        # Flag to compute running time for each epoch
        t = time.time()
        loss = train_func()
        avg_cost = loss
        print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(avg_cost),
        "time=", "{:.5f}".format(time.time() - t))
    mean_time.append(time.time() - t_start)

    if args.verbose:
        print("Testing model...")
    auc, ap = test_func(test_pos_edge_index, test_neg_edge_index)
    roc_score, ap_score = auc, ap
    mean_roc.append(roc_score)
    mean_ap.append(ap_score)

# if adaptive...
print('=' * 60)
print(args.adaptive)
print(args.model)
print('=' * 60)
    
# Report final results
print("\nTest results for", args.model,
      "model on", args.dataset, "on", args.task, "\n",
      "___________________________________________________\n")

print("AUC scores\n", mean_roc)
print("Mean AUC score: ", np.mean(mean_roc),
      "\nStd of AUC scores: ", np.std(mean_roc), "\n \n")

print("AP scores \n", mean_ap)
print("Mean AP score: ", np.mean(mean_ap),
      "\nStd of AP scores: ", np.std(mean_ap), "\n \n")

print("Running times\n", mean_time)
print("Mean running time: ", np.mean(mean_time),
      "\nStd of running time: ", np.std(mean_time), "\n \n")

Masking test edges...
torch.Size([4230, 4230]) torch.Size([4230, 4230])
Training...
Epoch: 0001 train_loss= 1.38648 time= 0.10868
Epoch: 0002 train_loss= 1.38561 time= 0.10802
Epoch: 0003 train_loss= 1.38427 time= 0.10416
Epoch: 0004 train_loss= 1.38224 time= 0.10644
Epoch: 0005 train_loss= 1.37883 time= 0.10904
Epoch: 0006 train_loss= 1.37403 time= 0.10700
Epoch: 0007 train_loss= 1.36621 time= 0.10521
Epoch: 0008 train_loss= 1.35347 time= 0.10379
Epoch: 0009 train_loss= 1.33695 time= 0.10874
Epoch: 0010 train_loss= 1.31319 time= 0.11110
Epoch: 0011 train_loss= 1.28783 time= 0.11022
Epoch: 0012 train_loss= 1.26631 time= 0.10572
Epoch: 0013 train_loss= 1.24334 time= 0.10555
Epoch: 0014 train_loss= 1.23844 time= 0.10499
Epoch: 0015 train_loss= 1.21380 time= 0.10956
Epoch: 0016 train_loss= 1.19232 time= 0.11307
Epoch: 0017 train_loss= 1.17893 time= 0.09137
Epoch: 0018 train_loss= 1.16915 time= 0.13221
Epoch: 0019 train_loss= 1.15351 time= 0.08407
Epoch: 0020 train_loss= 1.14040 time= 0.10

Epoch: 0178 train_loss= 0.57532 time= 0.10506
Epoch: 0179 train_loss= 0.58046 time= 0.10759
Epoch: 0180 train_loss= 0.58073 time= 0.10603
Epoch: 0181 train_loss= 0.59122 time= 0.10496
Epoch: 0182 train_loss= 0.58570 time= 0.10601
Epoch: 0183 train_loss= 0.58113 time= 0.10399
Epoch: 0184 train_loss= 0.59118 time= 0.11201
Epoch: 0185 train_loss= 0.58708 time= 0.09946
Epoch: 0186 train_loss= 0.58454 time= 0.08427
Epoch: 0187 train_loss= 0.57404 time= 0.10011
Epoch: 0188 train_loss= 0.57609 time= 0.11290
Epoch: 0189 train_loss= 0.57133 time= 0.11902
Epoch: 0190 train_loss= 0.56616 time= 0.10100
Epoch: 0191 train_loss= 0.57526 time= 0.09305
Epoch: 0192 train_loss= 0.59151 time= 0.10894
Epoch: 0193 train_loss= 0.57304 time= 0.10406
Epoch: 0194 train_loss= 0.56459 time= 0.12349
Epoch: 0195 train_loss= 0.57562 time= 0.10144
Epoch: 0196 train_loss= 0.56910 time= 0.09869
Epoch: 0197 train_loss= 0.58547 time= 0.11040
Epoch: 0198 train_loss= 0.57222 time= 0.10403
Epoch: 0199 train_loss= 0.57534 ti

Epoch: 0155 train_loss= 0.63532 time= 0.10505
Epoch: 0156 train_loss= 0.64934 time= 0.10295
Epoch: 0157 train_loss= 0.65327 time= 0.10183
Epoch: 0158 train_loss= 0.65149 time= 0.09759
Epoch: 0159 train_loss= 0.63310 time= 0.10307
Epoch: 0160 train_loss= 0.64331 time= 0.10198
Epoch: 0161 train_loss= 0.64418 time= 0.10815
Epoch: 0162 train_loss= 0.64916 time= 0.09699
Epoch: 0163 train_loss= 0.64109 time= 0.10073
Epoch: 0164 train_loss= 0.62254 time= 0.10223
Epoch: 0165 train_loss= 0.64554 time= 0.10159
Epoch: 0166 train_loss= 0.64265 time= 0.10479
Epoch: 0167 train_loss= 0.63670 time= 0.10811
Epoch: 0168 train_loss= 0.61805 time= 0.09200
Epoch: 0169 train_loss= 0.62159 time= 0.10450
Epoch: 0170 train_loss= 0.62767 time= 0.10000
Epoch: 0171 train_loss= 0.63510 time= 0.10106
Epoch: 0172 train_loss= 0.62547 time= 0.10148
Epoch: 0173 train_loss= 0.63187 time= 0.10087
Epoch: 0174 train_loss= 0.62630 time= 0.09981
Epoch: 0175 train_loss= 0.62348 time= 0.09897
Epoch: 0176 train_loss= 0.60956 ti

Epoch: 0131 train_loss= 0.69308 time= 0.10566
Epoch: 0132 train_loss= 0.68215 time= 0.09748
Epoch: 0133 train_loss= 0.68241 time= 0.08526
Epoch: 0134 train_loss= 0.67324 time= 0.11549
Epoch: 0135 train_loss= 0.68054 time= 0.10200
Epoch: 0136 train_loss= 0.69048 time= 0.10100
Epoch: 0137 train_loss= 0.66859 time= 0.10401
Epoch: 0138 train_loss= 0.66596 time= 0.10699
Epoch: 0139 train_loss= 0.69113 time= 0.10497
Epoch: 0140 train_loss= 0.67330 time= 0.10301
Epoch: 0141 train_loss= 0.66837 time= 0.11196
Epoch: 0142 train_loss= 0.66626 time= 0.10406
Epoch: 0143 train_loss= 0.67639 time= 0.10893
Epoch: 0144 train_loss= 0.66430 time= 0.09594
Epoch: 0145 train_loss= 0.67508 time= 0.10292
Epoch: 0146 train_loss= 0.66846 time= 0.10200
Epoch: 0147 train_loss= 0.67072 time= 0.10051
Epoch: 0148 train_loss= 0.67130 time= 0.10113
Epoch: 0149 train_loss= 0.66277 time= 0.09892
Epoch: 0150 train_loss= 0.66804 time= 0.10154
Epoch: 0151 train_loss= 0.67277 time= 0.10165
Epoch: 0152 train_loss= 0.65899 ti

Epoch: 0109 train_loss= 0.65999 time= 0.10699
Epoch: 0110 train_loss= 0.66954 time= 0.10143
Epoch: 0111 train_loss= 0.66734 time= 0.09268
Epoch: 0112 train_loss= 0.67264 time= 0.10201
Epoch: 0113 train_loss= 0.66954 time= 0.10506
Epoch: 0114 train_loss= 0.65834 time= 0.10400
Epoch: 0115 train_loss= 0.67859 time= 0.10505
Epoch: 0116 train_loss= 0.67184 time= 0.10300
Epoch: 0117 train_loss= 0.66635 time= 0.10702
Epoch: 0118 train_loss= 0.64132 time= 0.10101
Epoch: 0119 train_loss= 0.65639 time= 0.10699
Epoch: 0120 train_loss= 0.67372 time= 0.10400
Epoch: 0121 train_loss= 0.65079 time= 0.10172
Epoch: 0122 train_loss= 0.65354 time= 0.10110
Epoch: 0123 train_loss= 0.62485 time= 0.09823
Epoch: 0124 train_loss= 0.64014 time= 0.09542
Epoch: 0125 train_loss= 0.64454 time= 0.11056
Epoch: 0126 train_loss= 0.64544 time= 0.10060
Epoch: 0127 train_loss= 0.63194 time= 0.10999
Epoch: 0128 train_loss= 0.63523 time= 0.09500
Epoch: 0129 train_loss= 0.62447 time= 0.11000
Epoch: 0130 train_loss= 0.63002 ti

...

Epoch: 0041 train_loss= 0.81846 time= 0.10500
Epoch: 0042 train_loss= 0.81077 time= 0.10199
Epoch: 0043 train_loss= 0.80640 time= 0.10101
Epoch: 0044 train_loss= 0.82436 time= 0.08534
Epoch: 0045 train_loss= 0.80478 time= 0.10013
Epoch: 0046 train_loss= 0.80512 time= 0.09312
Epoch: 0047 train_loss= 0.80226 time= 0.10872
Epoch: 0048 train_loss= 0.76378 time= 0.10336
Epoch: 0049 train_loss= 0.79416 time= 0.10259
Epoch: 0050 train_loss= 0.77870 time= 0.09804
Epoch: 0051 train_loss= 0.78447 time= 0.10196
Epoch: 0052 train_loss= 0.78426 time= 0.09899
Epoch: 0053 train_loss= 0.75264 time= 0.10000
Epoch: 0054 train_loss= 0.77782 time= 0.10300
Epoch: 0055 train_loss= 0.76039 time= 0.10300
Epoch: 0056 train_loss= 0.77398 time= 0.11818
Epoch: 0057 train_loss= 0.75233 time= 0.11528
Epoch: 0058 train_loss= 0.74942 time= 0.10388
Epoch: 0059 train_loss= 0.75890 time= 0.11380
Epoch: 0060 train_loss= 0.74247 time= 0.10602
Epoch: 0061 train_loss= 0.75435 time= 0.10396
Epoch: 0062 train_loss= 0.74088 ti

Epoch: 0019 train_loss= 1.13586 time= 0.10239
Epoch: 0020 train_loss= 1.11643 time= 0.10144
Epoch: 0021 train_loss= 1.10543 time= 0.10200
Epoch: 0022 train_loss= 1.07510 time= 0.10571
Epoch: 0023 train_loss= 1.06530 time= 0.10140
Epoch: 0024 train_loss= 1.02926 time= 0.10055
Epoch: 0025 train_loss= 1.00873 time= 0.10405
Epoch: 0026 train_loss= 0.97912 time= 0.10378
Epoch: 0027 train_loss= 0.96028 time= 0.11695
Epoch: 0028 train_loss= 0.93392 time= 0.09500
Epoch: 0029 train_loss= 0.93071 time= 0.10505
Epoch: 0030 train_loss= 0.90286 time= 0.10395
Epoch: 0031 train_loss= 0.88007 time= 0.09807
Epoch: 0032 train_loss= 0.87201 time= 0.10421
Epoch: 0033 train_loss= 0.86934 time= 0.09793
Epoch: 0034 train_loss= 0.87320 time= 0.08517
Epoch: 0035 train_loss= 0.85074 time= 0.10012
Epoch: 0036 train_loss= 0.84356 time= 0.10168
Epoch: 0037 train_loss= 0.84561 time= 0.10596
Epoch: 0038 train_loss= 0.81656 time= 0.09899
Epoch: 0039 train_loss= 0.82489 time= 0.10105
Epoch: 0040 train_loss= 0.83482 ti

Epoch: 0199 train_loss= 0.54729 time= 0.10121
Epoch: 0200 train_loss= 0.54687 time= 0.10069
Testing model...
Masking test edges...
torch.Size([4230, 4230]) torch.Size([4230, 4230])
Training...
Epoch: 0001 train_loss= 1.38635 time= 0.10641
Epoch: 0002 train_loss= 1.38567 time= 0.10774
Epoch: 0003 train_loss= 1.38412 time= 0.10491
Epoch: 0004 train_loss= 1.38172 time= 0.10237
Epoch: 0005 train_loss= 1.37789 time= 0.10100
Epoch: 0006 train_loss= 1.37206 time= 0.09827
Epoch: 0007 train_loss= 1.36233 time= 0.09648
Epoch: 0008 train_loss= 1.34904 time= 0.09885
Epoch: 0009 train_loss= 1.33088 time= 0.11384
Epoch: 0010 train_loss= 1.30890 time= 0.10931
Epoch: 0011 train_loss= 1.28048 time= 0.10610
Epoch: 0012 train_loss= 1.25876 time= 0.10680
Epoch: 0013 train_loss= 1.23719 time= 0.10264
Epoch: 0014 train_loss= 1.23410 time= 0.11061
Epoch: 0015 train_loss= 1.20805 time= 0.10182
Epoch: 0016 train_loss= 1.19562 time= 0.10350
Epoch: 0017 train_loss= 1.17555 time= 0.10517
Epoch: 0018 train_loss= 1

Epoch: 0175 train_loss= 0.57169 time= 0.10654
Epoch: 0176 train_loss= 0.56381 time= 0.10524
Epoch: 0177 train_loss= 0.57049 time= 0.09954
Epoch: 0178 train_loss= 0.56262 time= 0.10096
Epoch: 0179 train_loss= 0.56534 time= 0.09549
Epoch: 0180 train_loss= 0.57335 time= 0.10639
Epoch: 0181 train_loss= 0.57017 time= 0.08143
Epoch: 0182 train_loss= 0.57063 time= 0.09890
Epoch: 0183 train_loss= 0.56703 time= 0.10021
Epoch: 0184 train_loss= 0.59291 time= 0.10947
Epoch: 0185 train_loss= 0.57957 time= 0.10581
Epoch: 0186 train_loss= 0.57153 time= 0.10178
Epoch: 0187 train_loss= 0.55173 time= 0.10244
Epoch: 0188 train_loss= 0.55793 time= 0.10045
Epoch: 0189 train_loss= 0.58349 time= 0.09202
Epoch: 0190 train_loss= 0.58498 time= 0.10265
Epoch: 0191 train_loss= 0.56778 time= 0.10191
Epoch: 0192 train_loss= 0.55887 time= 0.10633
Epoch: 0193 train_loss= 0.56953 time= 0.10493
Epoch: 0194 train_loss= 0.56427 time= 0.10529
Epoch: 0195 train_loss= 0.56143 time= 0.10536
Epoch: 0196 train_loss= 0.55912 ti

Epoch: 0151 train_loss= 0.64904 time= 0.13639
Epoch: 0152 train_loss= 0.63537 time= 0.09635
Epoch: 0153 train_loss= 0.61987 time= 0.08886
Epoch: 0154 train_loss= 0.63308 time= 0.09949
Epoch: 0155 train_loss= 0.63162 time= 0.11264
Epoch: 0156 train_loss= 0.62213 time= 0.10434
Epoch: 0157 train_loss= 0.61387 time= 0.10966
Epoch: 0158 train_loss= 0.63505 time= 0.11300
Epoch: 0159 train_loss= 0.62519 time= 0.11000
Epoch: 0160 train_loss= 0.60738 time= 0.12199
Epoch: 0161 train_loss= 0.62014 time= 0.11500
Epoch: 0162 train_loss= 0.61503 time= 0.12400
Epoch: 0163 train_loss= 0.61191 time= 0.10605
Epoch: 0164 train_loss= 0.60680 time= 0.10685
Epoch: 0165 train_loss= 0.60353 time= 0.11682
Epoch: 0166 train_loss= 0.62001 time= 0.11295
Epoch: 0167 train_loss= 0.61334 time= 0.10789
Epoch: 0168 train_loss= 0.60707 time= 0.10564
Epoch: 0169 train_loss= 0.60159 time= 0.10900
Epoch: 0170 train_loss= 0.59931 time= 0.11863
Epoch: 0171 train_loss= 0.59722 time= 0.08525
Epoch: 0172 train_loss= 0.60961 ti